In [6]:
!pip install opencv-contrib-python

In [7]:
!pip install cmake

In [8]:
!pip install face_recognition

In [ ]:
##############################################################################################################################################################################################
########################################################################

#actual code of face #################################################

In [10]:
import cv2
import numpy as np
import os

# Initialize the webcam
cap = cv2.VideoCapture(0)

# Load the pre-trained face classifier (Haar Cascade for face detection)
face_classifier = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Function to crop faces from the image
def face_cropped(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)  # Convert image to grayscale
    faces = face_classifier.detectMultiScale(gray, 1.3, 5)  # Detect faces

    if len(faces) == 0:
        return None  # No faces detected

    for (x, y, w, h) in faces:
        cropped_face = img[y:y+h, x:x+w]  # Crop the face from the image
        return cropped_face  # Return the cropped face

# Function to generate a dataset
def generate_dataset():
    img_id = 0
    class_name = input("Enter the name of the person: ")

    print(f"Collecting samples for {class_name}....")

    # Custom dataset directory path
    dataset_dir = r'C:\Users\ganji\Codes\Mini Proj\face_new'  # Change this to your preferred path

    # Ensure the dataset directory exists
    if not os.path.exists(dataset_dir):
        os.makedirs(dataset_dir)

    while True:
        ret, frame = cap.read()  # Read a frame from the webcam
        
        if not ret:
            print("Failed to capture image")
            break
        
        # Get the cropped face
        face = face_cropped(frame)
        
        if face is not None:
            img_id += 1
            face_resized = cv2.resize(face, (200, 200))  # Resize the face to 200x200
            cv2.imshow(f'Face Collection: {class_name}', face_resized)  # Show the cropped face

            # Save the image in the custom dataset path
            cv2.imwrite(f"{dataset_dir}/{class_name}_{img_id}.jpg", face_resized)
            
        # Stop after collecting 100 samples
        if img_id >= 700:
            print(f"Collected {img_id} samples for {class_name}.")
            break
        
        if cv2.waitKey(1) == 13:  # Stop on 'Enter' key
            break

    cap.release()  # Release the webcam
    cv2.destroyAllWindows()  # Close all OpenCV windows

# Call the function to start the image collection
generate_dataset()


Enter the name of the person:  Soldier_1


Collected 700 samples for Soldier_1.


In [11]:
import os
import cv2
import numpy as np
import yaml
from PIL import Image

def load_classes(yaml_file):
    """Load class names and their corresponding IDs from a YAML file."""
    with open(yaml_file, 'r') as file:
        data = yaml.safe_load(file)
    return data['classes']

def train_classifier(data_dir, yaml_file):
    # Load class names from the YAML file
    class_map = load_classes(yaml_file)
    
    # Get all image paths from the directory
    path = [os.path.join(data_dir, f) for f in os.listdir(data_dir) if f.endswith('.jpg')]

    if not path:
        print("No images found in the specified directory.")
        return
    
    faces = []
    ids = []

    for image_path in path:
        print(f"Processing image: {image_path}")  # Debug print to check which image is being processed
        img = Image.open(image_path).convert('L')  # Convert image to grayscale
        imageNp = np.array(img, 'uint8')  # Convert the image to a numpy array
        
        # Extract the class name from the filename
        class_name = os.path.basename(image_path).split("_")[0]  # Extract class name (Soldier, Criminal, etc.)
        
        # Assign an ID based on the class name from the YAML file
        if class_name in class_map:
            id = class_map[class_name]
        else:
            print(f"Unknown class name {class_name}. Skipping this image.")
            continue

        faces.append(imageNp)
        ids.append(id)

    if len(faces) == 0 or len(ids) == 0:
        print("No valid data for training. Ensure you have properly labeled images.")
        return

    ids = np.array(ids)
    
    # Train the classifier
    clf = cv2.face.LBPHFaceRecognizer_create()
    clf.train(faces, ids)
    
    # Save the trained classifier to a file
    clf.write("classifier.xml")
    print("Training completed and classifier saved to 'classifier.xml'.")

# Call the train_classifier function with the directory path where the images are saved and the path to the YAML file
train_classifier(r'C:\Users\ganji\Codes\Mini Proj\face_new', 'data.yaml')  # Replace with your actual dataset path and YAML file path


Processing image: C:\Users\ganji\Codes\Mini Proj\face_new\Soldier_1_1.jpg
Processing image: C:\Users\ganji\Codes\Mini Proj\face_new\Soldier_1_10.jpg
Processing image: C:\Users\ganji\Codes\Mini Proj\face_new\Soldier_1_100.jpg
Processing image: C:\Users\ganji\Codes\Mini Proj\face_new\Soldier_1_101.jpg
Processing image: C:\Users\ganji\Codes\Mini Proj\face_new\Soldier_1_102.jpg
Processing image: C:\Users\ganji\Codes\Mini Proj\face_new\Soldier_1_103.jpg
Processing image: C:\Users\ganji\Codes\Mini Proj\face_new\Soldier_1_104.jpg
Processing image: C:\Users\ganji\Codes\Mini Proj\face_new\Soldier_1_105.jpg
Processing image: C:\Users\ganji\Codes\Mini Proj\face_new\Soldier_1_106.jpg
Processing image: C:\Users\ganji\Codes\Mini Proj\face_new\Soldier_1_107.jpg
Processing image: C:\Users\ganji\Codes\Mini Proj\face_new\Soldier_1_108.jpg
Processing image: C:\Users\ganji\Codes\Mini Proj\face_new\Soldier_1_109.jpg
Processing image: C:\Users\ganji\Codes\Mini Proj\face_new\Soldier_1_11.jpg
Processing image

In [13]:
import cv2
import numpy as np
from PIL import Image
import os

def draw_boundary(img, classifier, scaleFactor, minNeighbors, color, text, clf):
    if img is None:
        print("Error: Camera frame not received")
        return img  # Return as-is if no valid frame is received
    
    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    features = classifier.detectMultiScale(gray_img, scaleFactor, minNeighbors)
     
    for (x, y, w, h) in features:
        cv2.rectangle(img, (x, y), (x + w, y + h), color, 2)
         
        id, pred = clf.predict(gray_img[y:y + h, x:x + w])
        confidence = int(100 * (1 - pred / 300))
        
        # Debug: Show predicted confidence
        print(f"Predicted confidence: {confidence}%")

        if confidence > 80:  # Increased confidence threshold
            if id == 1:
                cv2.putText(img, "Soldier", (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 1, cv2.LINE_AA)
            elif id == 2:
                cv2.putText(img, "Terrorist", (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 1, cv2.LINE_AA)
        else:
            # Handle low confidence case correctly
            cv2.putText(img, "UNKNOWN", (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 1, cv2.LINE_AA)
     
    return img

# loading classifier
faceCascade = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
 
clf = cv2.face.LBPHFaceRecognizer_create()
clf.read("classifier.xml")

# Initialize the camera
video_capture = cv2.VideoCapture(0)  # Ensure the correct camera index (0 is usually the default)

while True:
    ret, img = video_capture.read()
    if not ret:
        print("Failed to grab frame from the camera.")
        break  # Exit the loop if no frame is captured
    
    img = draw_boundary(img, faceCascade, 1.3, 6, (255, 255, 255), "Face", clf)
    cv2.imshow("Face Detection", img)
    
    if cv2.waitKey(1) == 13:  # Press Enter to exit
        break

video_capture.release()
cv2.destroyAllWindows()


Predicted confidence: 81%
Predicted confidence: 82%
Predicted confidence: 81%
Predicted confidence: 81%
Predicted confidence: 81%
Predicted confidence: 81%
Predicted confidence: 81%
Predicted confidence: 82%
Predicted confidence: 81%
Predicted confidence: 82%
Predicted confidence: 82%
Predicted confidence: 81%
Predicted confidence: 81%
Predicted confidence: 81%
Predicted confidence: 82%
Predicted confidence: 81%
Predicted confidence: 82%
Predicted confidence: 81%
Predicted confidence: 81%
Predicted confidence: 81%
Predicted confidence: 81%
Predicted confidence: 81%
Predicted confidence: 81%
Predicted confidence: 82%
Predicted confidence: 81%
Predicted confidence: 81%
Predicted confidence: 81%
Predicted confidence: 81%
Predicted confidence: 81%
Predicted confidence: 81%
Predicted confidence: 82%
Predicted confidence: 82%
Predicted confidence: 82%
Predicted confidence: 82%
Predicted confidence: 81%
Predicted confidence: 81%
Predicted confidence: 81%
Predicted confidence: 82%
Predicted co

In [10]:
import cv2
import numpy as np

def draw_boundary(img, classifier, scaleFactor, minNeighbors, color, text, clf):
    if img is None:
        print("Error: Camera frame not received")
        return img  # Return as-is if no valid frame is received
    
    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    features = classifier.detectMultiScale(gray_img, scaleFactor, minNeighbors)
     
    for (x, y, w, h) in features:
        cv2.rectangle(img, (x, y), (x + w, y + h), color, 2)
         
        id, pred = clf.predict(gray_img[y:y + h, x:x + w])
        confidence = int(100 * (1 - pred / 300))
        
        # Debug: Show predicted confidence
        print(f"Predicted confidence: {confidence}%")

        if confidence > 80:  # Increased confidence threshold
            if id == 1:
                cv2.putText(img, "Soldier", (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 1, cv2.LINE_AA)
            elif id == 2:
                cv2.putText(img, "Terrorist", (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 1, cv2.LINE_AA)
        else:
            # Handle low confidence case correctly
            cv2.putText(img, "UNKNOWN", (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 1, cv2.LINE_AA)
     
    return img

# Load the Haar Cascade and trained model
faceCascade = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
clf = cv2.face.LBPHFaceRecognizer_create()
clf.read("classifier.xml")

# Initialize both cameras
camera1 = cv2.VideoCapture(0)  # Inbuilt webcam
camera2 = cv2.VideoCapture(1)  # Wired webcam (check index if needed)

while True:
    # Capture frames from both cameras
    ret1, frame1 = camera1.read()
    ret2, frame2 = camera2.read()

    if not ret1 or not ret2:
        print("Error: Could not grab frames from one or both cameras.")
        break  # Exit the loop if a frame is not captured
    
    # Process frames with face detection
    frame1 = draw_boundary(frame1, faceCascade, 1.3, 6, (255, 255, 255), "Face", clf)
    frame2 = draw_boundary(frame2, faceCascade, 1.3, 6, (0, 255, 0), "Face", clf)
    
    # Display frames side by side
    combined_frame = cv2.hconcat([frame1, frame2])
    cv2.imshow("Face Detection - Camera 1 & Camera 2", combined_frame)
    
    if cv2.waitKey(1) == 13:  # Press Enter to exit
        break

# Release the cameras and close windows
camera1.release()
camera2.release()
cv2.destroyAllWindows()

Error: Could not grab frames from one or both cameras.
